### Tensor Flow - modelo de comparação

A biblioteca TensorFlow é uma estrutura de aumento de gradiente que usa algoritmos de aprendizagem baseados em árvore para implementar este modelo. Ele é distribuível e capaz de lidar com grandes volumes de dados com alto desempenho. TensorFlow é ideal para uma variedade de tarefas de aprendizado de máquina, como classificação, regressão e ranqueamento, graças à sua velocidade e eficiência.

### Instruções para uso local ou remoto (Google Colab ou VScode)

Aqui deixaremos brevemente um passo a passo para que você usuário seja capaz de executar o código localmente ou remotamente pelo seu google drive, podendo escolher a forma mais viável para seu uso e conhecimento.

##### Google Colab

1. Faça o upload do seu arquivo .ipynb para o Google Drive.
2. Abra o Google Colab em seu navegador.
3. Clique em "Arquivo" no menu superior e selecione "Abrir notebook".
4. Na guia "Upload", clique em "Procurar" e selecione o arquivo .ipynb que você enviou para o Google Drive.
5. Após selecionar o arquivo, clique em "Abrir".
6. Aguarde o carregamento do notebook no Google Colab.
7. Agora que você carregou o notebook no Google Colab, você pode fazer as alterações necessárias nos arquivos e caminhos para se adequar ao seu ambiente específico.

No notebook, a célula seguinte à essa contém as leituras dos arquivos CSV com o caminho do drive do criador desse notebook.
Comente as linhas que fazem referência aos arquivos locais e descomente as linhas que fazem referência ao Google Drive. Por exemplo:

- descomente as linhas que começam com # from google.colab import drive;
- comente as linhas que começam com tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv").

Certifique-se de que os arquivos CSV estejam localizados no diretório correto em seu ambiente virtual. Por exemplo, se você tiver uma pasta chamada "data" no mesmo diretório do notebook, coloque os arquivos CSV nessa pasta e ajuste seus nomes. Possivelmente os arquivos vão seguir o seguinte padrão, mesmo no seu drive:

tabela_Meta = pd.read_csv("/content/drive/MyDrive/NomeDaPastaDosArquivos/arquivo.csv)

Salve as alterações no notebook.
Agora você pode executar as células do notebook no Google Colab e as alterações nos arquivos e caminhos serão aplicadas ao seu ambiente do colab. Lembre-se de que você não precisará ter as bibliotecas necessárias instaladas em seu ambiente local para executar o código corretamente, dado que ao executar pela ferramenta do google essas dependências estarão aplicadas por padrão

##### VScode

1. Coloque os arquivos CSV dentro da pasta "data" desse notebook

No notebook, a célula seguinte à essa contém as leituras dos arquivos CSV com o caminho do drive do criador desse notebook.
Comente as linhas que fazem referência aos arquivos locais e descomente as linhas que fazem referência ao Google Drive. Por exemplo:

- descomente as linhas que começam com # from google.colab import drive;
- comente as linhas que começam com tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv").

Certifique-se de que os arquivos CSV estejam localizados no diretório correto em seu ambiente virtual. Por exemplo, se você tiver uma pasta chamada "data" no mesmo diretório do notebook, coloque os arquivos CSV nessa pasta e ajuste seus nomes. Possivelmente os arquivos vão seguir o seguinte padrão, mesmo no seu drive:

tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv")

Salve as alterações no notebook.
Agora você pode executar as células do notebook no VScode e as alterações nos arquivos e caminhos serão aplicadas ao seu ambiente do colab. Lembre-se de que você precisará ter as bibliotecas necessárias instaladas em seu ambiente local para executar o código corretamente:

Para fazer a instalação, basta abrir o terminal integrado e inserir o seguinte:

In [61]:
#pip install tensorflow keras pandas numpy scikit-learn keras-tuner

Feito isso vamos importar as bibliotecas e dependências usadas para nosso notebook.

In [62]:
import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense # type: ignore
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import kerastuner as kt
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error
from math import sqrt


A seguir, iremos importar as tabelas já limpas para podermos utilizar no modelo TensorFlow.

In [63]:
# # pegando arquivos csv do drive
# from google.colab import drive
# drive.mount('/content/drive')

# # Lendo os arquivos CSV
# tabela_Meta = pd.read_csv("/content/drive/MyDrive/Primeiro Ano/Módulo 3 - Modelo Preditivo Gazeta/Base de Dados Limpas/BASE INTELI_META-limpo.csv")
# tabela_Agosto = pd.read_csv("/content/drive/MyDrive/Primeiro Ano/Módulo 3 - Modelo Preditivo Gazeta/Base de Dados Limpas/tratada_BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.csv")

# para realizar o processo localmente, descomente as linhas abaixo e comente as linhas acima.
tabela_Meta = pd.read_csv("../data/dados_tratados/BASE INTELI_META-limpo.csv")
tabela_Agosto = pd.read_csv("../data/dados_tratados/tratada_BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.csv")

A seguir, é configurado o formato para exibir números sem casas decimais e realizada a agregação dos dados de 'Vl Liquido Final' com base em colunas de interesse, como 'Ano', 'Mês', 'Veiculo', etc.

In [ ]:
# Definindo formato de exibição
pd.set_option('display.float_format', '{:.0f}'.format)

# Agrupar e somar 'Vl Liquido Final' e 'Outra Coluna'
tabela_agosto_segmento = tabela_Agosto.groupby(['Ano', 'Mês', 'Veiculo', 'Origem', 'Segmento']).agg({
    'Vl Liquido Final': 'sum',
    # 'VL Tabela': 'sum'

}).reset_index()

# tabela_agosto_segmento['VL Tabela'] = tabela_agosto_segmento['VL Tabela'] / 1000
tabela_agosto_segmento.head()

As colunas categóricas são transformadas em valores numéricos utilizando LabelEncoder, e as variáveis dependentes e independentes são separadas.

In [65]:
categorical_columns = ['Veiculo', 'Origem', 'Segmento']
X = tabela_agosto_segmento.drop('Vl Liquido Final', axis=1)
y = tabela_agosto_segmento['Vl Liquido Final']

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le



Os dados são normalizados para garantir que todas as variáveis tenham a mesma escala, o que é importante para o treinamento do modelo.

In [66]:
# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=4)

A função build_model define a arquitetura da rede neural, com duas camadas densas (fully connected), e permite a variação de hiperparâmetros como número de neurônios e taxa de aprendizado.

In [67]:
# Função para construir o modelo
def build_model(hp):
    modelo = Sequential()
    modelo.add(Dense(units=hp.Int('units_1', min_value=32, max_value=128, step=32), 
                     activation='relu', 
                     input_dim=X_train.shape[1]))
    modelo.add(Dense(units=hp.Int('units_2', min_value=16, max_value=64, step=16), activation='relu'))
    modelo.add(Dense(1))
    
    modelo.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')), 
                   loss='mean_squared_error', 
                   metrics=['mae', 'mse'])
    return modelo


#### Keras Tuner

O Keras Tuner é uma biblioteca que ajuda a encontrar automaticamente os melhores hiperparâmetros para modelos de deep learning, otimizando o desempenho do modelo sem necessidade de ajuste manual. Ele permite testar diferentes combinações de parâmetros, como o número de camadas, unidades por camada, e taxa de aprendizado. Abaixo, mostramos como o Keras Tuner foi configurado para buscar os melhores hiperparâmetros no nosso modelo.

In [ ]:
# Configurar o Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=10,
    executions_per_trial=10,
    directory='my_dir',
    project_name='intro_to_kt'
)


O tuner é treinado com um subconjunto dos dados para encontrar os melhores hiperparâmetros. Após a busca, os melhores parâmetros são impressos.

In [ ]:
# Treinar o tuner
tuner.search(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32)

# Imprimir os melhores hiperparâmetros
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Melhores Parâmetros:")
print(f"Number of units in first layer: {best_hps.get('units_1')}")
print(f"Number of units in second layer: {best_hps.get('units_2')}")
print(f"Learning rate: {best_hps.get('learning_rate')}")


O modelo com os melhores hiperparâmetros é treinado em todo o conjunto de treinamento.

In [ ]:
# Construir o modelo com os melhores hiperparâmetros
best_model = tuner.get_best_models(num_models=1)[0]

# Treinar o modelo final
best_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=0)
# Avaliar o modelo no conjunto de teste
results = best_model.evaluate(X_test, y_test, verbose=1)

# Prever os valores no conjunto de teste
y_pred = best_model.predict(X_test)
y_pred = np.ravel(y_pred)


## Métricas

#### Erro Médio Absoluto (MAE - do inglês *Mean Absoluto Error*)

Como podemos observar, também calculamos o MAE, que é uma métrica que mede o quão longe, em média, as previsões de um modelo estão dos valores reais. Ele calcula a média das diferenças absolutas entre o que o modelo previu e o que realmente aconteceu. Quanto menor for o valor do MAE, mais precisas foram as previsões. Abaixo, mostramos como esse valor foi calculado no modelo do grupo:

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

#### Raiz do Erro Quadrático Médio (RMSE - do inglês *Root Mean Squared Error*)

Além disso, também calculamos o MSE que é uma métrica que avalia a precisão de um modelo ao calcular a média dos erros quadráticos entre os valores preditos e os reais. Ele eleva ao quadrado a diferença entre as previsões e os valores observados, penalizando erros maiores. Quanto menor o valor do MSE, melhor o desempenho do modelo. Abaixo, mostramos o cálculo dessa métrica no modelo do grupo:

In [ ]:
rmse = mean_squared_error(y_test, y_pred)
print(f"Root Mean Squared Error: {sqrt(rmse)}")

#### Coeficiente de Determinação (R²)

O Coeficiente de Determinação R² é uma métrica que indica quanto da variação dos dados o modelo consegue explicar. Ele mede o quão bem as previsões se ajustam aos dados reais, comparando com a média dos valores. A fórmula inclui uma subtração por 1, de modo que, quanto mais próximo de 1 for o valor de R², melhor o modelo explica a variação dos dados. No entanto, o R² sozinho não é suficiente para avaliar completamente o desempenho do modelo, sendo necessário considerar outras métricas como MAE e MSE. Abaixo, mostramos o cálculo dessa métrica no modelo do grupo:

In [ ]:
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

#### Erro Médio Absoluto Percentual (MAPE - do inglês Mean Absolute Percentage Error)

O MAPE é uma métrica utilizada para avaliar a precisão de modelos de regressão, calculando a média dos erros percentuais absolutos entre os valores reais e os valores preditos. Quanto menor o valor do MAPE, melhor o desempenho do modelo. Abaixo, apresentamos o cálculo desse indicador no modelo do grupo:

In [ ]:
mape = mean_absolute_percentage_error(y_test, y_pred)
print (f"MAPE: {mape}")

### Conlusões


Embora os resultados obtidos sejam positivos, é evidente que, em comparação com o modelo entregue na sprint 3 de desenvolvimento, o desempenho do TensorFlow ainda apresenta uma desvantagem em relação ao Random Forest Regressor, como mostrado a seguir:

Métrica | Random Forest Regressor | Tensor Flow Regressor
--- | --- | ---
R² | 91% | 1,87%
MAE | 10.417 | 31.219,27
RMSE | 31.757 | 108.716,55
MAPE | 394% | 2580%

A seguir, vamos analisar os resultados visualmente:

O gráfico compara os valores reais com as previsões do modelo. A linha roxa representa as previsões geradas pelo modelo, enquanto os pontos vermelhos correspondem aos valores reais. Quanto mais próximos os pontos estiverem da linha, melhor a qualidade das previsões.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='red', alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='purple', lw=3)  # Linha de identidade
plt.xlabel('Valores Reais')
plt.ylabel('Previsões')
plt.title('Comparação entre Valores Reais e Previsões - Modelo Tensor Flow')
plt.grid(True)
plt.show()

Com base nessas métricas, observamos que o Random Forest Regressor supera o LightGBM de maneira significativa. O R² do TensorFlow, com apenas 1%, indica que ele explica muito pouco da variabilidade dos dados em comparação ao Random Forest, que apresenta 91%. Além disso, o MAE e o RMSE do LightGBM são extremamente elevados, sugerindo que suas previsões são muito menos precisas do que as do Random Forest. Esses resultados deixam claro que o Random Forest é o modelo muito mais confiável e adequado para esse conjunto de dados.